# import

In [1]:
import json
import os
import re
from pprint import pprint

from soynlp.normalizer import repeat_normalize
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers

In [2]:
def dump_jsonl(data, output_path, append=False):
    mode = "a+" if append else "w"
    with open(output_path, mode, encoding="utf-8") as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + "\n")


def load_jsonl(input_path) -> list:
    data = []
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.rstrip("\n|\r")))
    return data

In [3]:
os.chdir('/workspace/TFTrainer/')

vocab_size = 20000

# load data

In [4]:
def processing(text):
    text = text.replace("\n", " ").strip()
    text = re.sub(r" {2,}", r" ", text)
    text = repeat_normalize(text, 3)
    text = re.sub(r"(.{3,}?)\1+", r"\1", text)
    text = re.sub(r"[^ ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z0-9]", r"", text)

    return text

In [5]:
data_json = load_jsonl('category.json')

data_txt = []
for sample in data_json:
    data_txt.append(sample['content'].strip())

# processing

with open("tokenizer_data.txt", "w", encoding='utf-8') as f:
    for line in data_txt:
        f.write(line + '\n')

In [6]:
data_txt[:10]

['1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품의 부패를 방지하기 위해 산소 차단 기능이 적용된 포장재는 이전까지 일본산 소재인 에틸렌비닐알코올로만 만들어졌다 에버켐텍은 천연 단백질을 이용한 신소재로 일본의 식품포장재 시장 독점 구조를 깬 셈이다 에버켐텍이 소재부품장비 강소기업으로 평가받는 이유다2 2001년 설립된 영창케미칼은 일본의 3대 수출규제 품목 중 하나였던 포토레지스트 소재 국산화를 선도했다 반도체 공정의 필수 소재인 포토레지스트는 빛으로 회로 모양을 찍어내는 작업을 할 때 웨이퍼 위에 균일하게 도포되는 액체를 말한다 영창케미칼의 포토레지스트는 경쟁력을 인정받아 현재 삼성전자 SK하이닉스 등 국내 반도체 회사에 공급된다일본 경제산업성이 2019년 7월 고순도 불화수소 플루오린폴리이미드 포토레지스트에 대한 수출규제를 시작하면서 촉발된 소부장 기술전쟁이 2년째를 맞는다 정부는 소부장 기업들이 혁신과 정부 지원정책이 맞물려 기술 자립도가 높아졌다고 평가한다 100대 핵심 품목 대일 의존도 급감중소벤처기업부와 산업통상자원부에 따르면 최근 2년 동안 소부장 관련 100대 핵심 품목에 대한 대일 의존도는 줄고 소부장 기업의 매출은 20 넘게 증가했다 중기부가 선정한 소부장 강소기업100에 포함돼 있는 에버켐텍과 영창케미칼 등의 성과를 보면 일본의 수출규제가 소부장 기업들이 성장하는 계기가 된 셈이다소부장 기술독립의 성과는 양과 질적인 면에서 함께 나타나고 있다 시가총액이 1조 원 이 넘는 소부장 관련 중소중견기업 수는 2019년 13개에서 올해 31개로 늘었다소부장 생태계 내에서 대기업과 중소기업 간 협업 사례도 늘었다 중기부에 따르면 대기업은 생산 라인을 개방해 중소기업이 신규 기술을 검증받을 수 있도록 했다 그러면서 대기업은 중소기업이 개발한 소재를 공급받고 있다 이에 따라 지난해 1분기 소부장 상장기업의 총매출액은 2019년 1분기 대비 201 증가했다 이는 상장기업 전체 평균 매출액 증가율보다 크게 높은 것이다 소

In [7]:
user_defined_symbols = ["<pad>", "<unk>", "<cls>", "<sep>", "<mask>", "<bos>", "<eos>", "<tsep>", "<unk0>", "<unk1>", "<unk2>", "<unk3>", "<unk4>", "<unk5>", "<unk6>", "<unk7>", "<unk8>", "<unk9>"]
unused_token_num = 100
unused_list = [f"<unused{i}>" for i in range(50)]
user_defined_symbols += unused_list

pprint(user_defined_symbols)

['<pad>',
 '<unk>',
 '<cls>',
 '<sep>',
 '<mask>',
 '<bos>',
 '<eos>',
 '<tsep>',
 '<unk0>',
 '<unk1>',
 '<unk2>',
 '<unk3>',
 '<unk4>',
 '<unk5>',
 '<unk6>',
 '<unk7>',
 '<unk8>',
 '<unk9>',
 '<unused0>',
 '<unused1>',
 '<unused2>',
 '<unused3>',
 '<unused4>',
 '<unused5>',
 '<unused6>',
 '<unused7>',
 '<unused8>',
 '<unused9>',
 '<unused10>',
 '<unused11>',
 '<unused12>',
 '<unused13>',
 '<unused14>',
 '<unused15>',
 '<unused16>',
 '<unused17>',
 '<unused18>',
 '<unused19>',
 '<unused20>',
 '<unused21>',
 '<unused22>',
 '<unused23>',
 '<unused24>',
 '<unused25>',
 '<unused26>',
 '<unused27>',
 '<unused28>',
 '<unused29>',
 '<unused30>',
 '<unused31>',
 '<unused32>',
 '<unused33>',
 '<unused34>',
 '<unused35>',
 '<unused36>',
 '<unused37>',
 '<unused38>',
 '<unused39>',
 '<unused40>',
 '<unused41>',
 '<unused42>',
 '<unused43>',
 '<unused44>',
 '<unused45>',
 '<unused46>',
 '<unused47>',
 '<unused48>',
 '<unused49>']


# train

In [9]:
from transformers import GPT2TokenizerFast, AutoTokenizer

In [10]:
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.NFKC()
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Metaspace()])
tokenizer.decoders = decoders.Metaspace()

trainer = trainers.BpeTrainer(
    vocab_size=vocab_size, 
    show_progress=True,
    special_tokens=user_defined_symbols,
)

tokenizer.train_from_iterator(data_txt, trainer=trainer)

In [11]:
!mkdir temp
tokenizer.model.save("temp")

['temp/vocab.json', 'temp/merges.txt']

# save according to form

In [12]:
tokenizer_for_load = GPT2TokenizerFast.from_pretrained("temp")  # 로드

tokenizer_for_load.pad_token = "<pad>"
tokenizer_for_load.unk_token = "<unk>"
tokenizer_for_load.cls_token = "<cls>"
tokenizer_for_load.sep_token = "<sep>"
tokenizer_for_load.mask_token = "<mask>"
tokenizer_for_load.bos_token = "<bos>"
tokenizer_for_load.eos_token = "<eos>"

special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
tokenizer_for_load.add_special_tokens(special_tokens_dict)

tokenizer_for_load.save_pretrained("tokenizer", legacy_format=False)

file temp/config.json not found
file temp/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/tokenizer.json')

in `tokenizer.json`  
```json
    "normalizer": {
        "type": "Sequence",
        "normalizers": [
            {
                "type": "NFKC"
            },
            {
                "type": "BertNormalizer",
                "clean_text": false,
                "handle_chinese_chars": false,
                "strip_accents": false,
                "lowercase": false
            }
        ]
    },
    "pre_tokenizer": {
        "type": "Sequence",
        "pretokenizers": [
            {
                "type": "Metaspace",
                "replacement": "▁",
                "add_prefix_space": true
            }
        ]
    },
    "post_processor": null,
    "decoder": {
        "type": "Metaspace",
        "replacement": "▁",
        "add_prefix_space": true
    },
```

in `tokenizer_config.json`  
```json
,
    "model_type": "gpt2"
```

rename `tokenizer_config.json` => `config.json`

In [13]:
t = AutoTokenizer.from_pretrained("tokenizer")


e = t("본 고안은 이러한 특성을 이용해 사용한다.")
print(e)
print(t.decode(e['input_ids']))

{'input_ids': [2782, 2568, 5271, 4714, 9027, 5190, 18833], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
본 고안은 이러한 특성을 이용해 사용한다
